In [1]:
import pandas as pd
import numpy as np
import os
import sys
import numbers
def apply_filter_query(df, filters=None):
	"""
	Use the DataFrame.query method to filter a table down to the
	desired rows.

	Parameters
	----------
	df : pandas.DataFrame
	filters : list of str or str, optional
		List of filters to apply. Will be joined together with
		' and ' and passed to DataFrame.query. A string will be passed
		straight to DataFrame.query.
		If not supplied no filtering will be done.

	Returns
	-------
	filtered_df : pandas.DataFrame

	"""
	if filters:
		if isinstance(filters, str):
			query = filters
		else:
			query = ' and '.join(filters)
		return df.query(query)
	else:
		return df
def _filterize(name, value):
	"""
	Turn a `name` and `value` into a string expression compatible
	the ``DataFrame.query`` method.

	Parameters
	----------
	name : str
		Should be the name of a column in the table to which the
		filter will be applied.

		A suffix of '_max' will result in a "less than" filter,
		a suffix of '_min' will result in a "greater than or equal to" filter,
		and no recognized suffix will result in an "equal to" filter.
	value : any
		Value side of filter for comparison to column values.

	Returns
	-------
	filter_exp : str

	"""
	if name.endswith('_min'):
		name = name[:-4]
		comp = '>='
	elif name.endswith('_max'):
		name = name[:-4]
		comp = '<'
	else:
		comp = '=='

	result = '{} {} {!r}'.format(name, comp, value)
	return result

def filter_table(table, filter_series, ignore=None):
	"""
	Filter a table based on a set of restrictions given in
	Series of column name / filter parameter pairs. The column
	names can have suffixes `_min` and `_max` to indicate
	"less than" and "greater than" constraints.

	Parameters
	----------
	table : pandas.DataFrame
		Table to filter.
	filter_series : pandas.Series
		Series of column name / value pairs of filter constraints.
		Columns that ends with '_max' will be used to create
		a "less than" filters, columns that end with '_min' will be
		used to create "greater than or equal to" filters.
		A column with no suffix will be used to make an 'equal to' filter.
	ignore : sequence of str, optional
		List of column names that should not be used for filtering.

	Returns
	-------
	filtered : pandas.DataFrame

	"""
	ignore = ignore if ignore else set()

	filters = [_filterize(name, val)
				for name, val in filter_series.iteritems()
				if not (name in ignore or
						(isinstance(val, numbers.Number) and
						np.isnan(val)))]

	return apply_filter_query(table, filters)


In [2]:
run_folder = "/home/da/semcog_urbansim/runs"
run_num = "run290"

In [3]:
hdf = pd.HDFStore(os.path.join(run_folder, '%s.h5'%run_num), 'r')


In [4]:
hdf.keys()

['/base/annual_employment_control_totals',
 '/base/annual_household_control_totals',
 '/base/annual_relocation_rates_for_households',
 '/base/annual_relocation_rates_for_jobs',
 '/base/base_job_space',
 '/base/building_sqft_per_job',
 '/base/building_types',
 '/base/buildings',
 '/base/counties',
 '/base/crime_rates',
 '/base/employed_workers_rate',
 '/base/employment_sectors',
 '/base/events_addition',
 '/base/events_deletion',
 '/base/group_quarters',
 '/base/group_quarters_control_totals',
 '/base/households',
 '/base/jobs',
 '/base/land_use_types',
 '/base/large_areas',
 '/base/parcels',
 '/base/persons',
 '/base/poi',
 '/base/refiner_events',
 '/base/schools',
 '/base/semmcds',
 '/base/target_vacancies',
 '/base/target_vacancies_mcd',
 '/base/transit_stops',
 '/base/travel_data',
 '/base/travel_data_2030',
 '/base/zones',
 '/base/zoning',
 '/2050/base_job_space',
 '/2050/buildings',
 '/2050/dropped_buildings',
 '/2050/group_quarters',
 '/2050/households',
 '/2050/jobs',
 '/2050/pa

### Household controls

In [5]:
region_ct = hdf['/base/annual_household_control_totals']
region_hh = hdf["/2050/households"]
region_p = hdf["/2050/persons"]
region_target = pd.read_csv('/home/da/share/urbansim/RDF2050/model_inputs/base_tables/remi_pop_total.csv', index_col='large_area_id')

In [6]:
max_cols = region_ct.columns[
	region_ct.columns.str.endswith("_max") & (region_ct == -1).any(axis=0)
]
region_ct[max_cols] = region_ct[max_cols].replace(-1, np.inf)
region_ct[max_cols] += 1

In [7]:
region_hh.index = region_hh.index.astype(int)
region_p.index = region_p.index.astype(int)

In [8]:
region_ct_2050 = region_ct.loc[2050]
region_simulated_hh_2050 = [filter_table(region_hh, region_ct_2050.iloc[i], 'total_number_of_households').shape[0] for i in range(region_ct_2050.shape[0])]

In [9]:
region_ct.loc[2050, 'region_simulated_hh_2050'] = pd.Series(region_simulated_hh_2050, index=region_ct_2050.index)

In [13]:
region_ct.loc[2050, 'abs_diff'] = (region_ct.loc[2050, 'region_simulated_hh_2050']  - region_ct.loc[2050, 'total_number_of_households'] ).abs()
region_ct.loc[2050, 'ratio'] = region_ct.loc[2050, 'region_simulated_hh_2050']  / region_ct.loc[2050, 'total_number_of_households'] 

In [15]:
region_ct.loc[2050]

,persons_min,age_of_head_max,large_area_id,total_number_of_households,age_of_head_min,workers_max,cars_max,income_max,children_min,cars_min,persons_max,race_id,income_min,workers_min,children_max,region_simulated_hh_2050,abs_diff,ratio
year,,,,,,,,,,,,,,,,,,
2050,2,18.0,3,3,5,2.0,1.0,31704,0,0,3.0,3,0,1,1.0,3.0,0.0,1.000000
2050,1,25.0,3,34,18,1.0,1.0,31704,0,0,2.0,1,0,0,1.0,33.0,1.0,0.970588
2050,1,25.0,3,223,18,2.0,1.0,31704,0,0,2.0,1,0,1,1.0,223.0,0.0,1.000000
2050,1,25.0,3,309,18,1.0,2.0,31704,0,1,2.0,1,0,0,1.0,309.0,0.0,1.000000
2050,1,25.0,3,4,18,1.0,2.0,64016,0,1,2.0,1,31705,0,1.0,4.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050,4,inf,161,324,65,3.0,3.0,1534131,0,2,5.0,4,113191,2,1.0,321.0,3.0,0.990741
2050,4,inf,161,38,65,inf,inf,113190,0,3,5.0,4,64017,3,1.0,38.0,0.0,1.000000
2050,5,inf,161,6,65,1.0,3.0,1534131,0,2,6.0,4,113191,0,1.0,6.0,0.0,1.000000


In [16]:
region_ct.loc[2050].sort_values(by='abs_diff', ascending=False).head(20)

,persons_min,age_of_head_max,large_area_id,total_number_of_households,age_of_head_min,workers_max,cars_max,income_max,children_min,cars_min,persons_max,race_id,income_min,workers_min,children_max,region_simulated_hh_2050,abs_diff,ratio
year,,,,,,,,,,,,,,,,,,
2050,1,65.0,5,12220,35,1.0,1.0,31704,0,0,2.0,2,0,0,1.0,11530.0,690.0,0.943535
2050,1,65.0,5,7196,35,1.0,2.0,31704,0,1,2.0,2,0,0,1.0,6805.0,391.0,0.945664
2050,1,65.0,5,5457,35,2.0,2.0,64016,0,1,2.0,2,31705,1,1.0,5814.0,357.0,1.065421
2050,1,65.0,5,4536,35,2.0,2.0,31704,0,1,2.0,2,0,1,1.0,4860.0,324.0,1.071429
2050,4,inf,161,2767,65,2.0,inf,113190,0,3,5.0,3,64017,1,1.0,2519.0,248.0,0.910372
2050,3,inf,5,1650,65,3.0,3.0,113190,0,2,4.0,4,64017,2,1.0,1431.0,219.0,0.867273
2050,2,65.0,5,3273,35,1.0,1.0,31704,0,0,3.0,2,0,0,1.0,3066.0,207.0,0.936755
2050,2,inf,125,4790,65,2.0,3.0,1534131,0,2,3.0,1,113191,1,1.0,4973.0,183.0,1.038205
2050,1,inf,125,16553,65,1.0,2.0,31704,0,1,2.0,1,0,0,1.0,16385.0,168.0,0.989851


In [18]:
# diff by ratio
region_ct.loc[2050]['abs_diff'].sum()/region_ct.loc[2050]["total_number_of_households"].sum()

0.016768794454517035

### MCD totals

In [19]:
mcd_total = pd.read_csv('/home/da/share/urbansim/RDF2050/model_inputs/base_tables/mcd_totals_2020_2050_nov15.csv', index_col='mcd')

In [37]:
hdf['/base/households'].loc[1581462]

age_of_head                 57
cars                         4
ten                          1
large_area_id              147
persons                      4
income                  250018
workers                      4
valp                    500000
building_id            6095163
race_id                      1
children                     0
blkgrp            261635808001
household_type              17
Name: 1581462, dtype: int64

In [21]:
p50 = hdf["/2050/parcels"]
b50 = hdf["/2050/buildings"]
b50 = b50.join(p50.semmcd, on='parcel_id')
region_hh = region_hh.join(b50.semmcd, on='building_id')

In [23]:
region_hh.building_id.isin(b50.index).sum()

2026432

In [24]:
region_hh.shape

(2056262, 14)

In [104]:
# number of hh dont have building_id
region_hh[region_hh.building_id == -1].shape[0]

29830

In [105]:
# percentage
region_hh[region_hh.building_id == -1].shape[0]/ region_hh.shape[0]

0.014506906221094393

In [125]:
simulated_mcd_hh = region_hh.semmcd.fillna(-1).value_counts()

In [160]:
diff = simulated_mcd_hh - mcd_total['2050']
mcd_total_vs_simulated = (diff)[diff.abs().sort_values(ascending=False).index]
mcd_total_vs_simulated.index = mcd_total_vs_simulated.index.astype(int)

In [161]:
mcd_total_vs_simulated.fillna(0).astype(int).head(50)

2172   -11188
2252   -10643
2073    -2816
6142    -1214
2040      392
2290      293
2075      208
2280      162
2230      157
5        -144
2015      126
2090      114
2020      108
2235      106
2060      104
2240       90
2150       82
2045       80
2260       75
2035       73
2160       71
2210       68
2080       64
6155       61
2005       56
4005      -53
2085       52
2030       51
2165       47
2205       41
3025      -37
2025       36
2100       34
2135       33
2115       31
2110       29
2300       28
2265       27
2120       25
7090       22
2175       21
5060       21
1105      -21
3135      -19
2185       18
2180       17
2050       17
2200       14
2125       12
2130       11
dtype: int64

In [141]:
simulated_mcd_hh.loc[2252.0]

13

In [140]:
hdf["/base/semmcds"].loc[2252]

city_name        Southfield Twp
county_id                   125
large_area_id               125
Name: 2252, dtype: object

In [ ]:
b50 = b50.join(p50.semmcd, on='parcel_id')

,parcel_id,sp_filter,improvement_value,residential_units,non_residential_sqft,sqft_per_unit,stories,building_type_id,market_value,year_built,land_area,sqft_price_res,sqft_price_nonres,event_id,mcd_model_quota,semmcd
building_id,,,,,,,,,,,,,,,,
1000016,1360532,0,259397.618497,1.0,0.000000,1902.0,1.00,81,408325.317919,1985,1902,1980.717757,0.0,0,0.0,1135
1000035,1360480,0,309378.424954,1.0,0.000000,2724.0,2.00,81,450462.631927,1984,1362,1660.675548,0.0,0,0.0,1135
1000096,1360565,0,357748.589895,1.0,0.000000,3047.0,2.00,81,503442.774278,1985,1524,1600.271016,0.0,0,0.0,1135
1000122,1196635,0,353370.309037,1.0,0.000000,2345.0,2.00,81,462366.828645,1973,1172,524.166244,0.0,0,0.0,1130
1000140,1360430,0,311671.231693,1.0,0.000000,2498.0,1.75,81,437260.955582,1985,1427,542.346511,0.0,0,0.0,1135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9044692,7037325,0,0.000000,1.0,0.000000,1123.0,1.00,81,0.000000,2050,0,1122.382758,0.0,0,0.0,7035
9044693,7036019,0,0.000000,4.0,0.000000,1930.4,1.00,81,0.000000,2050,0,848.514944,0.0,0,0.0,7035
9044694,7036594,0,0.000000,2.0,448.008990,2000.0,1.00,82,0.000000,2050,0,107.888180,0.0,0,0.0,7035


In [147]:
b50[b50.semmcd == 2252].shape

(9, 16)

In [150]:
# checking other year
year= 2020
b_temp = hdf['/%s/buildings'%year]
p_temp = hdf['/%s/parcels'%year]
b_temp = b_temp.join(p_temp.semmcd, on='parcel_id')
(b_temp.semmcd==2252).sum()

9

In [152]:
mcd_total.loc[2252]

2020    10458.0
2021       17.0
2022    10390.0
2023    10406.0
2024    10442.0
2025    10462.0
2026    10492.0
2027    10509.0
2028    10530.0
2029    10538.0
2030    10579.0
2031    10600.0
2032    10634.0
2033    10675.0
2034    10687.0
2035    10712.0
2036    10724.0
2037    10728.0
2038    10728.0
2039    10722.0
2040    10718.0
2041    10710.0
2042    10707.0
2043    10700.0
2044    10690.0
2045    10682.0
2046    10674.0
2047    10670.0
2048    10668.0
2049    10663.0
2050    10656.0
Name: 2252, dtype: float64